Modify the buildParseTree and evaluate functions to handle boolean statements (and, or, and not). Remember that “not” is a unary operator, so this will complicate your code somewhat.

In [1]:
#implemeting a stack LIFO: LAST IN, FIRST OUT 
class Stack:
    
    def __init__(self):
        self.items = []
    #check if the stack is empty and return true or false
    def isEmpty(self):
        return self.items == []
    #push, adds an item to the stack  
    def push(self, item):
        self.items.append(item)
    #pop, removes the most recently added item from stack
    def pop(self):
        return self.items.pop()
    #returns the top item from the stack but does not remove it
    def peek(self):
        return self.items[len(self.items)-1]
    #return the size of the stack: the numbers of items
    def size(self):
        return len(self.items)

In [2]:
class BinaryTree:
    # returns a tree, root node has no children yet
    #    where r is the value (key) for that node
    def __init__(self, root_obj):
        self.key = root_obj
        self.left_child = None
        self.right_child = None  

    def insert_left(self, new_val):
        # if no left child, create new node instance
        if self.left_child == None:
            self.left_child = BinaryTree(new_val)
        # if left child already exists
        else:
            t = BinaryTree(new_val)
            # make current node child of new node
            t.left_child = self.left_child
            # set t.left-child as t
            self.left_child = t

    def insert_right(self, new_val):
        # if no right child, create new node instance
        if self.right_child == None:
            self.right_child = BinaryTree(new_val)
        # if right child already exists
        else:
            t = BinaryTree(new_val)
            # make current node child of new node
            t.right_child = self.right_child
            # set t.right-child as t
            self.right_child = t

    def get_root_val(self):
        return self.key

    def set_root_val(self, obj):
        self.key = obj
    
    def get_left_child(self):
        return self.left_child
    
    def get_right_child(self):
        return self.right_child
    
    def __repr__(self):
        return str(self.key)

In modifieded build_parse_tree the operation 'and' 'or' behave in the way as arithmetic operations. However when the operation is 'not' which is a unary operation, we solve this issue by inserting an empty left child with key set to 'None'
and setting the current_node on the right child as it will hold the value of the following operand.

When we traverse the tree, we ignore the nodes that holds no value (like 'None'). So in the case where we have 'not' operation, we always ignore the left child.

In [3]:
def build_parse_tree(fp_exp):
    fp_list = fp_exp.split()
    p_stack = Stack()
    e_tree = BinaryTree('')
    p_stack.push(e_tree)
    current_tree = e_tree

    for i in fp_list:
        
        # if input is a '('
        if i == '(':
            # add new node as left child of current node
            current_tree.insert_left('')
            # push current tree to stack
            p_stack.push(current_tree)
            # descend to left child
            current_tree = current_tree.get_left_child()
        
        # if input is a  number
        elif i not in ['and', 'or', ')', 'not']:
            # set root val of current node to the boolean equivalent of the string
            # if string is 'True we set the value to boolean True
            if i == 'True':
                current_tree.set_root_val(True)
            # otherwise we set the value to boolean False
            else:
                current_tree.set_root_val(False)
            # set parent = stack.pop()
            parent = p_stack.pop()
            # return to parent
            current_tree = parent
        
        # if input is operation 'and, 'or'
        elif i in ['and', 'or']:
            # set root val of current_tree to the operator
            current_tree.set_root_val(i)
            # add new node as right child of current node
            current_tree.insert_right('')
            # push current tree to stack
            p_stack.push(current_tree)
            # descend to right child (set ct to rc)
            current_tree = current_tree.get_right_child()            
        
        # if input is operation 'not'
        elif i == 'not':
            # set root val of current_tree to the operator 
            current_tree.set_root_val(i)
            # push current tree to stack
            p_stack.push(current_tree)
            # Not is unary operation so we add a left child that has a None value 
            # that will be ignored in later flow
            current_tree.insert_left(None)
            # Add new node as right child of current node
            current_tree.insert_right('')
            # descend to right child 
            current_tree = current_tree.get_right_child()
        
        # if input is a ')'
        elif i == ')':
            # go to parent of current node
            current_tree = p_stack.pop()

        # in case strange character appears
        else:
            raise ValueError
    return e_tree

Inorder recursive traversal, helps to print the tree with the correct order of the operands and operation in between. Inorder prints the left side first, operation, and right side

In [4]:
def inorder(tree):
    if tree:
        inorder(tree.get_left_child())
        print(tree.get_root_val(), end=" ")
        inorder(tree.get_right_child())

In [5]:
pt = build_parse_tree(" ( ( True and False ) or not False ) ")  # every token needs a space around i

In [6]:
inorder(pt)

True and False or None not False 

In [7]:
pt = build_parse_tree(" ( not ( True and False ) ) ")  # every token needs a space around i

In [8]:
inorder(pt)

None not True and False  

In [9]:
pt = build_parse_tree(" ( not True ) and ( not False ) ")  # every token needs a space around i

In [10]:
inorder(pt)

None not True and None not False  

In [11]:
pt = build_parse_tree(" ( not True ) ")  # every token needs a space around i

In [12]:
inorder(pt)

None not True  

In [13]:
pt = build_parse_tree(" ( ( True and False ) or not False ) ")  # every token needs a space around i

In [14]:
inorder(pt)

True and False or None not False 

Evaluate funtion takes a tree as an input and has four main flows:
    1- when we have both left and child nodes to the current tree root node, we recursively evaulate both left and right sides with the operation in between which is the tree root node value
    2- In the case where we don't have a right child (None) and the tree only has a left child
        * if the current tree node value holds operation then we recursively evaulate only the left side
        * otherwise if the current tree node holds no value, then we set the left child as the new current tree node (as there's no importance to evaluate with no operation)
    3- If the current tree node has the operation 'not', here we recursively evaluate the the right side only, as in 'not' operation we don't have a left child
    4- When the current tree node is a leaf node we only return the value of it (True or False)

In [15]:
import operator

# ( True or False )
def evaluate2(parse_tree):
    # operator dictionary
    # operator.add is a Fx that adds X, Y (operator.add(X, Y))
    opers = {"and": operator.and_, "or": operator.or_, "not": operator.not_}

    left_c = parse_tree.get_left_child()
    right_c = parse_tree.get_right_child()
    
    # checking scenario when there's left child but the key value is None
    # which is usually the case when we have a Not operation where the left
    # child is created with None value
    if (left_c is not None) and (left_c.get_root_val() == None):
        left_c = None

    # if current node has children (is not a leaf)
    # here we evaluate the left and right with the operation in between
    # this flow works when operation is 'or' ' and'
    if left_c and right_c:
        # look up operator (root val of current node) and assign to 'fn'
        fn = opers[parse_tree.get_root_val()]
        # executes fn on the left and right children (operands)
        return fn(evaluate2(left_c), evaluate2(right_c))  # two recursive calls to evaluate left and right subtrees
    
    # this flow is when we don't have a right child and only left child exists
    # in the tree. if the parent node is operation we only evaluate the left side
    # otherwise if the node value is not operation then we set the left child as
    # the parent because there's no purpose to evaluate with an empty right side.
    elif left_c:
        if parse_tree.get_root_val() in ["and", "or", "not"]:
            fn = opers[parse_tree.get_root_val()]
            return fn(evaluate2(left_c))
        else:
            parse_tree = left_c
            return evaluate2(parse_tree)
        
    # if the node has key set to operation 'not' we apply not on the recursion of the right child only
    # because 'Not' node doesn't have any left child to evaluate with
    elif parse_tree.get_root_val() == 'not':
        fn = opers[parse_tree.get_root_val()]
        return fn(evaluate2(right_c))
    
    # case where recusion ends # if lef_c is None then False and true is still a false so go to base case
    else:
        return parse_tree.get_root_val()

In [16]:
pt = build_parse_tree(" ( not False ) ")  # every token needs a space around i

In [17]:
print(evaluate2(pt))

True


In [18]:
pt = build_parse_tree(" ( not True ) ")  # every token needs a space around i

In [19]:
print(evaluate2(pt))

False


In [20]:
pt = build_parse_tree(" ( not True ) and ( not False) ")  # every token needs a space around i

In [21]:
print(evaluate2(pt))

False


In [22]:
pt = build_parse_tree(" ( ( True and False ) or not False ) ")  # every token needs a space around i

In [23]:
print(evaluate2(pt))

True
